# Importação das bibliotecas

In [4]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 5.3 MB/s 


In [5]:
import zipfile

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SpatialDropout1D, LSTM, Dense, Embedding, Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
tf.random.set_seed(42)

# Carregando o dataset

In [8]:
df = pd.read_csv('/content/drive/MyDrive/ProjetoFinal_ProcessText/datasets/NoStem_StopwordKeep_dataset.csv')
df

,sent_rating,processed_text_KeepStopwords,data
0,negativo,Bom em questão de som ela é realmente boa no e...,2021-06-09 00:00:00
1,negativo,O produto não responde corretamente aos comand...,2020-11-26 00:00:00
2,negativo,Quem compra uma caixa de som pensa normalmente...,2021-01-05 00:00:00
3,negativo,A funcionalidade é muito boa Comprei as lâmpad...,2021-06-16 00:00:00
4,positivo,Excelente assistente som muito bom e limpo a A...,2020-12-10 00:00:00
...,...,...,...
3805,positivo,A mídia não pôde ser carregada Gostei de tudo ...,2021-07-06 00:00:00
3806,positivo,Bom dia comprei a echo dot preta e o cabo da t...,2021-05-04 00:00:00
3807,positivo,Amei a Alexa ela é incrível Porém fiquei chate...,2021-05-18 00:00:00
3808,positivo,Atendeu minhas expectativas Reconhece voz ente...,2021-01-09 00:00:00


# Processamento dos dados

## Codificando os textos

Para posterior construção da camada de embedding



In [9]:
df['processed_text_KeepStopwords'] = df['processed_text_KeepStopwords'].apply(lambda x: x.lower())

In [10]:
reviews = df['processed_text_KeepStopwords'].values

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(reviews)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(reviews)
X = pad_sequences(encoded_docs, maxlen=100)
X.shape

(3810, 100)


## Pegando os pesos das palavras a partir do Word2Vec

Para posterior atribuição aos pesos da camada de embedding

In [3]:
# Download do Word2Vec CBOW de 100 vetores do repositório do NILC - USP
!wget --no-check-certificate \
http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s100.zip \
-O /tmp/cbow_s100.zip

--2022-04-04 02:23:07--  http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s100.zip
Connecting to 143.107.183.175:22980... connected.
HTTP request sent, awaiting response... 200 OK
Length: 326003567 (311M) [application/octet-stream]
Saving to: ‘/tmp/cbow_s100.zip’

/tmp/cbow_s100.zip  100%[===================>] 310.90M  8.20MB/s    in 46s     

2022-04-04 02:23:53 (6.78 MB/s) - ‘/tmp/cbow_s100.zip’ saved [326003567/326003567]



In [11]:
# Extração do arquivo .zip
with zipfile.ZipFile('/tmp/cbow_s100.zip', 'r') as zip_ref:
    zip_ref.extractall('/tmp/cbow')

In [12]:
# Lendo o embedding
embeddings_index = {}
f = open('/tmp/cbow/cbow_s100.txt')
for line in f:
    values = line.split()
    word = values[0]
    if len(values[1:]) > 100:
      coefs = np.asarray(values[2:], dtype='float32')
      embeddings_index[word] = coefs
    else:
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 929595 word vectors.


In [13]:
# Extraindo nossas palavras a partir das presentes no embedding
embedding_matrix = np.zeros((vocab_size + 1, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [14]:
embedding_matrix.shape

(7895, 100)

## Label encoding de y

In [15]:
y = df['sent_rating']
print(np.unique(y))
y = LabelEncoder().fit_transform(y)
np.unique(y)

['negativo' 'positivo']


array([0, 1])

## Divisão em Treino e teste

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [17]:
print(X_train.shape, X_test.shape)

(3048, 100) (762, 100)


# Construção dos modelos

## Inicializando o modelo base

In [18]:
model_base = Sequential()

model_base.add(Embedding(input_dim=vocab_size + 1, 
                         output_dim=100,
                         weights=[embedding_matrix],
                         input_length = 100,
                         mask_zero=True))

model_base.add(LSTM(100,input_shape=(100,),activation='tanh',))
model_base.add(Dense(10, activation='relu'))
model_base.add(Dropout(0.3))

model_base.add(Dense(1, activation='sigmoid'))
model_base.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          789500    
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 10)                1010      
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 870,921
Trainable params: 870,921
Non-trainable params: 0
_________________________________________________________________


## Compilando o modelo

In [19]:
metrics = ['acc','Precision','Recall']

model_base.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = metrics
)

## Treinando o modelo base

In [20]:
es = EarlyStopping(monitor='val_loss', mode='auto', patience = 2, verbose=1)

results = model_base.fit(
 X_train, y_train,
 epochs= 50,
 batch_size = 200,
 validation_split=0.2,
 callbacks = [es]
)

Epoch 1/50
13/13 [==============================] - 18s 689ms/step - loss: 0.5532 - acc: 0.8236 - precision: 0.8537 - recall: 0.9566 - val_loss: 0.4279 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 2/50
13/13 [==============================] - 6s 458ms/step - loss: 0.4459 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.4090 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 3/50
13/13 [==============================] - 6s 453ms/step - loss: 0.4028 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.3904 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 4/50
13/13 [==============================] - 6s 447ms/step - loss: 0.3640 - acc: 0.8507 - precision: 0.8519 - recall: 0.9981 - val_loss: 0.3663 - val_acc: 0.8508 - val_precision: 0.8632 - val_recall: 0.9808
Epoch 5/50
13/13 [==============================] - 6s 453ms/step - loss: 0.3019 - acc: 0.8757 - precision: 0.8862 - recall: 0.9797 - val_loss:

## Teste do modelo base

In [21]:
test_result = model_base.evaluate(X_test,  y_test, verbose=2)

24/24 - 1s - loss: 0.3484 - acc: 0.8727 - precision: 0.8955 - recall: 0.9611 - 647ms/epoch - 27ms/step


In [22]:
precision = test_result[2]
recall = test_result[3]
f1_score = (2*recall*precision)/(recall+precision)
print('F1-Score:', f1_score)

F1-Score: 0.9271224715616889


## Otimização

### Inicializando função de otimização do modelo

In [23]:
def model_builder(hp):
  model_new = Sequential()

  model_new.add(Embedding(input_dim=vocab_size + 1, 
                          output_dim=100,
                          weights=[embedding_matrix],
                          input_length = 100,
                          mask_zero=True))
  
  hp_units1 = hp.Int('units', min_value=5, max_value=155, step=5)
  model_new.add(LSTM(hp_units1,input_shape=(100,),activation='tanh',))

  hp_units2 = hp.Int('units2', min_value=5, max_value=100, step=5)
  model_new.add(Dense(hp_units2, activation='relu'))

  hp_units3 = hp.Float('units3', min_value=0.1, max_value=0.9, step=0.1)
  model_new.add(Dropout(hp_units3))

  model_new.add(Dense(1, activation='sigmoid'))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4,1e-5])

  model_new.compile(
      optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss = "binary_crossentropy",
      metrics = metrics
      )

  return model_new

### Inicializando o otimizador

In [24]:
tuner = kt.Hyperband(model_builder,
                     objective=kt.Objective("val_acc", direction="max"),
                     max_epochs=10,
                     factor=3,
                     directory='/content/drive/MyDrive/ProjetoFinal_ProcessText/',
                     project_name='tune_LSTM_8')

In [25]:
stop_early = EarlyStopping(monitor='val_loss', mode='auto', patience = 2)

### Busca pelo modelo otimizado

In [26]:
tuner.search(X_train, y_train, epochs=50,  batch_size = 200, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 01m 29s]
val_acc: 0.854098379611969

Best val_acc So Far: 0.8622950911521912
Total elapsed time: 00h 17m 02s
INFO:tensorflow:Oracle triggered exit


### Treino do modelo otimizado

In [29]:
model_tuned = tuner.hypermodel.build(best_hps)
model_tuned.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          789500    
                                                                 
 lstm_2 (LSTM)               (None, 55)                34320     
                                                                 
 dense_4 (Dense)             (None, 75)                4200      
                                                                 
 dropout_2 (Dropout)         (None, 75)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 76        
                                                                 
Total params: 828,096
Trainable params: 828,096
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
history = model_tuned.fit(X_train, y_train, batch_size = 200, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_acc']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
13/13 [==============================] - 15s 657ms/step - loss: 0.6255 - acc: 0.7408 - precision: 0.8503 - recall: 0.8438 - val_loss: 0.4641 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 2/50
13/13 [==============================] - 6s 471ms/step - loss: 0.4423 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.4140 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 3/50
13/13 [==============================] - 6s 465ms/step - loss: 0.4049 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.3979 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 4/50
13/13 [==============================] - 7s 539ms/step - loss: 0.3711 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.3755 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 5/50
13/13 [==============================] - 6s 468ms/step - loss: 0.3051 - acc: 0.8589 - precision: 0.8598 - recall: 0.9966 - val_loss:

### Repetição do treino para uma quantidade ótima de épocas


In [31]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch, batch_size = 200, validation_split=0.2)

Epoch 1/16
13/13 [==============================] - 13s 588ms/step - loss: 0.6117 - acc: 0.7912 - precision: 0.8536 - recall: 0.9108 - val_loss: 0.4590 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 2/16
13/13 [==============================] - 6s 436ms/step - loss: 0.4345 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.4126 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 3/16
13/13 [==============================] - 6s 436ms/step - loss: 0.3975 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.3998 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 4/16
13/13 [==============================] - 6s 436ms/step - loss: 0.3629 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.3837 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 5/16
13/13 [==============================] - 6s 440ms/step - loss: 0.3037 - acc: 0.8597 - precision: 0.8599 - recall: 0.9976 - val_loss:

### Teste do modelo otimizado

In [32]:
eval_result = hypermodel.evaluate(X_test, y_test)

24/24 [==============================] - 1s 28ms/step - loss: 0.6564 - acc: 0.8661 - precision: 0.8814 - recall: 0.9720


In [33]:
precision = eval_result[2]
recall = eval_result[3]
f1_score = (2*recall*precision)/(recall+precision)
print('F1-Score:', f1_score)

F1-Score: 0.9244444557448964


## Salvando as predições junto ao dataframe original

In [34]:
y_pred = []
for predict in hypermodel.predict(X_test):
  if predict > 0.5:
    y_pred.append(1)
  else:
    y_pred.append(0)

In [35]:
_, predict_df, _, _ = train_test_split(df, y, test_size=0.2, random_state=42)
predict_df.insert(1, 'predict', y_pred)
predict_df.to_csv('/content/drive/MyDrive/ProjetoFinal_ProcessText/predict_datasets/LSTM_predict.csv')